# Ejemplo 2: Creación de contenido publicitario multilingue
En el presente ejemplo se implementará un sistema multi agente haciendo uso del patron de Workflow Secuencial. Por tal motivo, se crearán los siguientes agentes que  tendrán como input inicial un mensaje del usuario:

- WriterAgent: Crea un post publicitario en base a la descripcion de un producto
- TranslatorAgent: Genera la versión en Inglés del post publicitario
- MarketingAgent: Agrega emojis a la publicación para hacerla más atractiva

In [1]:
# Load environment variables

from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from dataclasses import dataclass

from autogen_core import (
    MessageContext,
    RoutedAgent,
    SingleThreadedAgentRuntime,
    TopicId,
    TypeSubscription,
    message_handler,
    type_subscription,
)
from autogen_core.models import ChatCompletionClient, SystemMessage, UserMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient

In [3]:
# Defining models for Task and TaskResponse.

@dataclass
class Message:
    content: str

In [4]:
# Topics
writer_topic_type = "WriterAgent"
translator_topic_type = "TranslatorAgent"
marketing_topic_type = "MarketingAgent"

In [5]:
# Defining Writer Agent
@type_subscription(topic_type=writer_topic_type)
class WriterAgent(RoutedAgent):
    def __init__(self, model_client: ChatCompletionClient) -> None:
        super().__init__("Writer agent.")
        self._system_message = SystemMessage(
            content=(
                "Eres un escritor experto en la creación de publicaciones atractivas para redes sociales dado un "
                "texto con la descripcion del producto."
                "Longitud recomendada: entre 150 y 300 palabras. "
                "Evita tecnicismos complejos (salvo que el tema lo requiera). "
                "No agregues emojis ni hashtags; otro agente se encargará de eso."
            )
        )
        self._model_client = model_client

    @message_handler
    async def handle_intermediate_text(self, message: Message, ctx: MessageContext) -> None:
        prompt = f"Descripcion del producto:\n\n{message.content}"

        llm_result = await self._model_client.create(
            messages=[self._system_message, UserMessage(content=prompt, source=self.id.key)],
            cancellation_token=ctx.cancellation_token,
        )
        response = llm_result.content
        assert isinstance(response, str)
        print(f"{'-'*80}\n{self.id.type}:\n{response}")

        await self.publish_message(Message(response), topic_id=TopicId(translator_topic_type, source=self.id.key))

In [6]:
# Translator Agent
@type_subscription(topic_type=translator_topic_type)
class TranslatorAgent(RoutedAgent):
    def __init__(self, model_client: ChatCompletionClient) -> None:
        super().__init__("Translator agent.")
        self._system_message = SystemMessage(
            content=(
                "Eres un traductor profesional con experiencia en redes sociales y marketing digital. "
                "Dado un post de redes sociales sobre un producto o servicio, debes devolver el texto original y su traducción a inglés. "
                "Asegúrate de que la traducción suene natural y fluida para hablantes nativos del idioma de destino. "
                "Evita traducciones literales; adapta el contenido si es necesario para que tenga impacto en la audiencia."
            )
        )
        self._model_client = model_client

    @message_handler
    async def handle_intermediate_text(self, message: Message, ctx: MessageContext) -> None:
        prompt = f"Post en idioma original:\n\n{message.content}"

        llm_result = await self._model_client.create(
            messages=[self._system_message, UserMessage(content=prompt, source=self.id.key)],
            cancellation_token=ctx.cancellation_token,
        )
        response = llm_result.content
        assert isinstance(response, str)
        print(f"{'-'*80}\n{self.id.type}:\n{response}")

        await self.publish_message(Message(response), topic_id=TopicId(marketing_topic_type, source=self.id.key))

In [8]:
#Marketing Agent
@type_subscription(topic_type=marketing_topic_type)
class MarketingAgent(RoutedAgent):
    def __init__(self, model_client: ChatCompletionClient) -> None:
        super().__init__("Marketing agent.")
        self._system_message = SystemMessage(
            content=(
                "Eres un experto en marketing digital y redes sociales. "
                "Recibirás una publicación en español e inglés y debes mejorarlas para hacerlas más atractiva "
                "para plataformas como Instagram, Facebook y Twitter. "
                "Agrega emojis relevantes para hacer el texto más visual y atractivo. "
                "Añade hashtags populares relacionados con el tema. "
                "No alteres la esencia del mensaje."
            )
        )
        self._model_client = model_client

    @message_handler
    async def handle_intermediate_text(self, message: Message, ctx: MessageContext) -> None:
        prompt = f"Publicaciones para redes sociales:\n{message.content}."
        llm_result = await self._model_client.create(
            messages=[self._system_message, UserMessage(content=prompt, source=self.id.key)],
            cancellation_token=ctx.cancellation_token,
        )
        response = llm_result.content
        assert isinstance(response, str)
        print(f"{'-'*80}\n{self.id.type}:\n{response}")


In [9]:
# Subscribing in the runtime

import os

model_client = OpenAIChatCompletionClient(
    model="gpt-4o-mini",
    api_key=os.environ.get("OPENAI_API_KEY")
)

runtime = SingleThreadedAgentRuntime()

await WriterAgent.register(runtime, type=writer_topic_type, factory=lambda: WriterAgent(model_client=model_client))

await TranslatorAgent.register(runtime, type=translator_topic_type, factory=lambda: TranslatorAgent(model_client=model_client))

await MarketingAgent.register(runtime, type=marketing_topic_type, factory=lambda: MarketingAgent(model_client=model_client))

AgentType(type='MarketingAgent')

In [ ]:
# Runing workflow
runtime.start()

await runtime.publish_message(
    Message(content="Bibicleta Bike3000 util para subir montañas y deportes de aventura."),
    topic_id=TopicId(writer_topic_type, source="default"),
)

await runtime.stop_when_idle()

--------------------------------------------------------------------------------
WriterAgent:
Descubre la libertad de explorar los senderos más desafiantes con la Bicicleta Bike3000, tu compañera ideal para aventuras en la montaña. Diseñada para aquellos que aman la naturaleza y el desafío, esta bicicleta se destaca por su robustez y durabilidad. Su estructura resistente te brinda la confianza necesaria para conquistar cualquier terreno, desde empinadas pendientes hasta caminos irregulares.

La Bike3000 cuenta con un sistema de suspensión eficaz que absorbe los impactos, proporcionando un viaje suave y cómodo, incluso en las rutas más difíciles. Su diseño aerodinámico y ligero facilita el manejo en ascensos pronunciados, permitiéndote alcanzar nuevas alturas sin esfuerzo adicional.

Ideal para los amantes de los deportes al aire libre, esta bicicleta no solo es perfecta para el ciclismo de montaña, sino que también se adapta a distintas actividades de aventura. Ya sea que estés subiend